In [0]:
#https://www.leafly.com/strains/lists/condition/nausea

!pip install requests-html > /dev/null

In [0]:
from pygments import highlight
from pygments.lexers import HtmlLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import HTML
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
from urllib.request import urlopen
from requests.adapters import HTTPAdapter
import requests
import re
import json
import numpy as np

In [0]:
def pretty_print_html(html_string):
  html_string = BeautifulSoup(html_string, 'html.parser').prettify()
  pygments_css = pygments_css=HtmlFormatter().get_style_defs('.highlight')
  data = highlight(html_string, HtmlLexer(), HtmlFormatter())
  return display(HTML(f'<style>{pygments_css}</style>{data}'))

In [27]:
!wget 'https://www.leafly.com/strains/lists/condition/nausea'

--2020-03-06 03:46:46--  https://www.leafly.com/strains/lists/condition/nausea
Resolving www.leafly.com (www.leafly.com)... 104.18.202.76, 104.18.203.76, 2606:4700::6812:cb4c, ...
Connecting to www.leafly.com (www.leafly.com)|104.18.202.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘nausea.1’

nausea.1                [  <=>               ] 180.81K   564KB/s    in 0.3s    

2020-03-06 03:46:48 (564 KB/s) - ‘nausea.1’ saved [185153]



In [0]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get('https://www.leafly.com/strains/lists/condition/nausea')

In [29]:
r

<Response [200]>

In [30]:
r.html.html

'<!DOCTYPE html><html lang="en"><head><link rel="preload" as="font" href="https://public.leafly.com/fonts/gt-america/gt-america-standard-thin.woff2" type="font/woff2" crossorigin="anonymous"/><link rel="preload" as="font" href="https://public.leafly.com/fonts/gt-america/gt-america-standard-medium.woff2" type="font/woff2" crossorigin="anonymous"/><link rel="preload" as="font" href="https://public.leafly.com/fonts/gt-america/gt-america-condensed-medium.woff2" type="font/woff2" crossorigin="anonymous"/><link rel="dns-prefetch" href="https://public.leafly.com"/><link rel="dns-prefetch" href="https://public.leafly.com"/><script type="text/javascript" src="https://public.leafly.com/scripts/split-10.10.0.min.js" defer=""></script><script type="text/javascript" src="https://public.leafly.com/cephalopod/dist/vendors.87e2f9ad0784ea148b7c.js" defer=""></script><script type="text/javascript" src="https://public.leafly.com/cephalopod/dist/helpers.c20c4673027656787dc7.js" defer=""></script><script t

In [31]:
pretty_print_html(r.html.html)

In [0]:
#naming our html document
document = r.html

In [33]:
#finding each strain name
strain_name = document.find('.strain-tile')
strain_name

[<Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/ocd'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/sour-strawberry'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/recon'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/urkle-train-haze'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/sour-blueberry'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/verde-electric'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/nepalese'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/blueberry-jack'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/rug-burn-og'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/lambo-og'>,
 <Element 'a' class=('strain-tile', 'justify-start',

In [0]:
#selecting the first strain name 
first_strain_name = strain_name[0]

In [35]:
#pretty print of the strain name
pretty_print_html(first_strain_name.html)

In [36]:
#pinpointed the text of the strain name!
first_strain_name.find('.strain-tile__name', first = True).text

'OCD'

In [37]:
#figured out how to get the text only
first_strain_name.find('span', first = True).text

'Aroused'

In [38]:
#pulling the attribute of the link only
first_strain_name.attrs['href']

'/strains/ocd'

In [0]:
#getting ready to create the dataframe
strain_list = []
for strain in strain_name:
  name = strain.find('.strain-tile__name')[0].text
  if strain.find('span'):
    affect = strain.find('span')[0].text
  else:
    affect = np.nan
  link = strain.attrs['href']
  strain_list.append({'Name':name, 'Affect':affect, 'Links': link})

In [41]:
df = pd.DataFrame(strain_list)
df

,Name,Affect,Links
0,OCD,Aroused,/strains/ocd
1,Sour Strawberry,Euphoric,/strains/sour-strawberry
2,Recon,Euphoric,/strains/recon
3,Urkle Train Haze,Happy,/strains/urkle-train-haze
4,Sour Blueberry,NaN,/strains/sour-blueberry
5,Verde Electric,Creative,/strains/verde-electric
6,Nepalese,Happy,/strains/nepalese
7,Blueberry Jack,Hungry,/strains/blueberry-jack
8,Rug Burn OG,Aroused,/strains/rug-burn-og
9,Lambo OG,Euphoric,/strains/lambo-og


In [42]:
df['Links'] = df['Links'].apply(lambda x: '<a href="http://www.leafly.com/{0}">Learn More'.format(x))

HTML(df.to_html(escape=False))

,Name,Affect,Links
0,OCD,Aroused,Learn More
1,Sour Strawberry,Euphoric,Learn More
2,Recon,Euphoric,Learn More
3,Urkle Train Haze,Happy,Learn More
4,Sour Blueberry,NaN,Learn More
5,Verde Electric,Creative,Learn More
6,Nepalese,Happy,Learn More
7,Blueberry Jack,Hungry,Learn More
8,Rug Burn OG,Aroused,Learn More
9,Lambo OG,Euphoric,Learn More


In [43]:
df

,Name,Affect,Links
0,OCD,Aroused,"<a href=""http://www.leafly.com//strains/ocd"">L..."
1,Sour Strawberry,Euphoric,"<a href=""http://www.leafly.com//strains/sour-s..."
2,Recon,Euphoric,"<a href=""http://www.leafly.com//strains/recon""..."
3,Urkle Train Haze,Happy,"<a href=""http://www.leafly.com//strains/urkle-..."
4,Sour Blueberry,NaN,"<a href=""http://www.leafly.com//strains/sour-b..."
5,Verde Electric,Creative,"<a href=""http://www.leafly.com//strains/verde-..."
6,Nepalese,Happy,"<a href=""http://www.leafly.com//strains/nepale..."
7,Blueberry Jack,Hungry,"<a href=""http://www.leafly.com//strains/bluebe..."
8,Rug Burn OG,Aroused,"<a href=""http://www.leafly.com//strains/rug-bu..."
9,Lambo OG,Euphoric,"<a href=""http://www.leafly.com//strains/lambo-..."


In [0]:
df.to_csv('nausea_strain_list')